1. `Installing Dependencies`
2. `Setup Folders` for Collection
3. `Preprocess` Data and `Create Labels and Features`
4. Build and `Train` LSTM Neural Network
5. Make `Predictions` based on Trained Model
6. `Save/Load` Weights
7. `Evaluate` Model using Confusion Matrix and Accuracy
8. `Live Test` the Model Accuracy

1. `Installing Dependencies`


In [2]:
import os
import torch
import numpy as np
# visible_devices = os.environ.get("CUDA_VISIBLE_DEVICES")
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1" # [-1, 0]
# print(f"CUDA_VISIBLE_DEVICES: {visible_devices}")
# print(torch.cuda.is_available())

2. `Setup Folders` for Collection

In [3]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data') 

# Actions that we try to detect
actions = np.array(['ako',  'bakit', 'F', 'hi', 'hindi', 'ikaw',  'kamusta', 'L', 'maganda', 'magandang umaga', 'N', 'O', 'oo', 'P', 'salamat'])

# number of videos we want to use for training
no_sequences = 17 # edit this to change the number of videos used for training

# Videos are going to be 30 frames in length
sequence_length = 30

3. `Preprocess` Data and `Create Labels/Features`

In [5]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

label_map = {label:num for num, label in enumerate(actions)}
print(label_map)

sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

X = np.array(sequences)
print('np.array(sequences).shape: ', np.array(sequences).shape)
print('np.array(labels).shape: ',np.array(labels).shape)

y = to_categorical(labels).astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
print('y_test shape:', y_test.shape)

{'ako': 0, 'bakit': 1, 'F': 2, 'hi': 3, 'hindi': 4, 'ikaw': 5, 'kamusta': 6, 'L': 7, 'maganda': 8, 'magandang umaga': 9, 'N': 10, 'O': 11, 'oo': 12, 'P': 13, 'salamat': 14}
np.array(sequences).shape:  (1845, 30, 1662)
np.array(labels).shape:  (1845,)
y_test shape: (185, 15)


4. Build and `Train` LSTM Neural Network

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard
from keras.layers import Conv1D

log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [8]:
# MODEL design and implementation
model = Sequential()
model.add(Conv1D(64, kernel_size=3, activation='relu', input_shape=(30,1662)))
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [9]:
model.fit(X_train, y_train, epochs=1200, callbacks=[tb_callback])

Epoch 1/1200
52/52 [==============================] - 7s 56ms/step - loss: 2.7139 - categorical_accuracy: 0.0584
Epoch 2/1200
52/52 [==============================] - 3s 59ms/step - loss: 2.7162 - categorical_accuracy: 0.0819
Epoch 3/1200
52/52 [==============================] - 3s 57ms/step - loss: 2.6818 - categorical_accuracy: 0.0633
Epoch 4/1200
52/52 [==============================] - 3s 56ms/step - loss: 2.6795 - categorical_accuracy: 0.1024
Epoch 5/1200
52/52 [==============================] - 3s 55ms/step - loss: 2.7027 - categorical_accuracy: 0.0880
Epoch 6/1200
52/52 [==============================] - 3s 58ms/step - loss: 2.6576 - categorical_accuracy: 0.0940
Epoch 7/1200
52/52 [==============================] - 3s 55ms/step - loss: 2.5015 - categorical_accuracy: 0.1289
Epoch 8/1200
52/52 [==============================] - 3s 55ms/step - loss: 2.4931 - categorical_accuracy: 0.1422
Epoch 9/1200
52/52 [==============================] - 3s 56ms/step - loss: 2.2809 - categorical_

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 28, 64)            319168    
                                                                 
 lstm (LSTM)                 (None, 28, 64)            33024     
                                                                 
 lstm_1 (LSTM)               (None, 28, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 15)                4

5. Make `Predictions` based on Trained Model


In [11]:
res = model.predict(X_test)
actions[np.argmax(res[2])]

6/6 [==============================] - 1s 22ms/step


'maganda'

In [12]:
actions[np.argmax(y_test[2])]

'maganda'

6. `Save/Load` Weights


In [14]:
model.save('fsl1.h5')

In [ ]:
del model
model.load_weights('action.h5')

7. `Evaluate` Model using Confusion Matrix and Accuracy

In [19]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

yhat = model.predict(X_test)
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [21]:
print(multilabel_confusion_matrix(ytrue, yhat))
print(accuracy_score(ytrue, yhat))

8. `Live Test` the Model Accuracy

In [24]:
from scipy import stats

colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [ ]:
plt.figure(figsize=(18,18))
plt.imshow(prob_viz(res, actions, image, colors))

In [26]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti